# Prep 1: steps to install the packages

In [3]:
# !pip install py_entitymatching
# !pip install scipy
# !pip install numpy
# !pip install pandas

# [NEED MODIFICATION: Modify this cell to point to the file location]
# Prep 2: enter the file location on your harddisk

In [1]:
table_a = './tableA_meta'
table_b = './tableB_imdb'
candidate_set = './candidate_set'
prediction_set = './prediction_list'

# Prep 3: reading the files into pandas dataframe

In [2]:
import pandas as pd

dfa = pd.read_csv(table_a)
dfb = pd.read_csv(table_b)
dfc = pd.read_csv(candidate_set)
dfp = pd.read_csv(prediction_set)

# Module: debug_blocker
# Description: debug the blocking rule using the below script to ensure you are not dropping true matches
# Note: You need to run Prep 1 and 2 in order to run this

In [3]:
# Example input format:
# Format of table_a:
# _id, attribute1, attribute2, ....., attributen

# Format of table_b:
# _id, attribute1, attribute2, ....., attributen

# Format of candidate_set
# A_id,B_id
# where A_id is _id from table_a and B_id is the _id column value from table_b

In [4]:
import py_entitymatching as em

def run_debug_blocker(table_a, table_b, table_a_key, table_b_key, candidate_set):
    dfl = em.read_csv_metadata(table_a, key=table_a_key)
    dfr = em.read_csv_metadata(table_b, key=table_b_key)

    # reading the candidate set and adding key
    dfcand = pd.read_csv(candidate_set)
    dfcand.drop_duplicates(inplace=True)
    dfcand.to_csv('cand_set_with_index.csv', index_label='id')

    dfcset = em.read_csv_metadata('cand_set_with_index.csv', key='id', ltable=dfl, 
                                  rtable=dfr, fk_ltable='A_id', fk_rtable='B_id')

    # running debug blocker to identify the records in A x B \ C
    debug_file = em.debug_blocker(dfcset, dfl, dfr)
    
    return debug_file

In [5]:
debug_file = run_debug_blocker(table_a, table_b, '_id', '_id', candidate_set)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [6]:
debug_file

,_id,ltable__id,rtable__id,ltable_title,ltable_rating,ltable_runtime,ltable_genres,ltable_director,ltable_starring,ltable_countries,...,rtable_rating,rtable_runtime,rtable_genres,rtable_director,rtable_starring,rtable_countries,rtable_languages,rtable_writers,rtable_production company,rtable_Up_System
0,0,2636,86,Harry Potter and the Deathly Hallows: Part I,PG-13,146 min,"Action,Adventure,Mystery,Fantasy",David Yates,"Daniel Radcliffe,Emma Watson,Rupert Grint","USA,UK",...,PG-13,138 min,"Adventure, Family, Fantasy",David Yates,"Daniel Radcliffe, Emma Watson, Rupert Grint, Brendan Gleeson",UK,English,"Michael Goldenberg (screenplay), J.K. Rowling (novel)","Warner Bros., Heyday Films, Cool Music",CM
1,1,149,1177,The Lives of Others,R,137 min,"Drama,Thriller",Florian Henckel von Donnersmarck,"Martina Gedeck,Sebastian Koch,Ulrich Mhe",Germany,...,PG-13,103 min,"Action, Adventure, Thriller",Florian Henckel von Donnersmarck,"Johnny Depp, Angelina Jolie, Paul Bettany, Timothy Dalton",USA,English,"Florian Henckel von Donnersmarck (screenplay), Christopher McQuarrie (screenplay)","GK Films, Spyglass Entertainment, Birnbaum/Barber",CM
2,2,1095,49,The Hunger Games: Catching Fire,PG-13,146 min,"Action,Adventure,Sci-Fi,Drama,Thriller",Francis Lawrence,"Jennifer Lawrence,Josh Hutcherson,Liam Hemsworth,Philip Seymour Hoffman",USA,...,PG-13,123 min,"Action, Adventure, Sci-Fi",Francis Lawrence,"Jennifer Lawrence, Josh Hutcherson, Liam Hemsworth, Woody Harrelson",USA,English,"Peter Craig (screenplay), Danny Strong (screenplay)","Color Force, Lionsgate",CM
3,3,917,33,Finding Dory,PG,97 min,"Adventure,Comedy,Animation,Family",Andrew Stanton,"Albert Brooks,Ellen DeGeneres",USA,...,G,100 min,"Animation, Adventure, Comedy","Andrew Stanton, Lee Unkrich","Albert Brooks, Ellen DeGeneres, Alexander Gould, Willem Dafoe",USA,English,"Andrew Stanton (original story by), Andrew Stanton (screenplay by)","Pixar Animation Studios, Walt Disney Pictures",CM
4,4,529,78,Harry Potter and the Prisoner of Azkaban,PG,141 min,"Adventure,Mystery,Fantasy,Family",Alfonso Cuarn,"Daniel Radcliffe,Rupert Grint","USA,UK",...,PG,153 min,"Adventure, Family, Fantasy",David Yates,"Daniel Radcliffe, Emma Watson, Rupert Grint, Michael Gambon",UK,English,"Steve Kloves (screenplay), J.K. Rowling (novel)","Warner Bros., Heyday Films",CM
5,5,2372,358,Spanking the Monkey,Unrated,100 min,"Drama,Comedy",David O. Russell,"Alberta Watson,Jeremy Davies",USA,...,R,138 min,"Crime, Drama",David O. Russell,"Christian Bale, Amy Adams, Bradley Cooper, Jennifer Lawrence",USA,English,"Eric Warren Singer, David O. Russell","Columbia Pictures, Annapurna Pictures, Atlas Entertainment",CM
6,6,1563,1203,The Rainmaker,PG-13,135 min,"Drama,Thriller,Crime",Francis Ford Coppola,"Danny DeVito,Matt Damon",USA,...,R,162 min,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Diane Keaton, Andy Garcia, Talia Shire",USA,English,"Mario Puzo, Francis Ford Coppola","Paramount Pictures, Zoetrope Studios",CM
7,7,3204,2212,The Sword in the Stone,G,79 min,"Adventure,Fantasy,Comedy,Animation,Family,Musical",Wolfgang Reitherman,"Karl Swenson,Rickie Sorensen,Sebastian Cabot",USA,...,G,78 min,"Animation, Adventure, Comedy",Wolfgang Reitherman,"Phil Harris, Eva Gabor, Sterling Holloway, Scatman Crothers",USA,English,"Larry Clemmons (story), Vance Gerry (story)",Walt Disney Productions,CM
8,8,3086,1246,A Few Good Men,R,138 min,"Drama,Mystery,Thriller,Crime",Rob Reiner,"Demi Moore,Jack Nicholson,Tom Cruise",USA,...,PG-13,114 min,"Comedy, Drama, Romance",Rob Reiner,"Michael Douglas, Annette Bening, Martin Sheen, Michael J. Fox",USA,English,Aaron Sorkin,"Universal Pictures, Castle Rock Entertainment, Wildwood Enterprises",CM
9,9,3150,962,10 Years,R,100 min,"Drama,Comedy,Romance",Jamie Linden,"Channing Tatum,Chris Pratt,Jenna Dewan",USA,...,PG-13,108 min,"Drama, Romance, War",Lasse Hallstrm,"Channing Tatum, Amanda Seyfried, Richard Jenkins, Henry Thomas",USA,English,"Jamie Linden (screenplay), Nicholas Sparks (novel)","Screen Gems, R

# Module: estimate_precision_recall
# Description: the below code helps you get an estimation of P/R on the candidate set.

In [7]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

delta = .05
Z = norm.ppf(1 - (delta / 2))

def estimate_PR(labeled_pairs, reduced_cands, predicted_matches):
    '''
    labeled_pairs - a pandas dataframe with schema id1,id2,label
                    Note label needs to be Boolean

    reduced_cands - a pandas dataframe with schema id1,id2
    predicted_matches - a pandas dataframe with schema id1,id2
    
    return:
        ( (recall lower bound, recall upper bound), (precision lower bound, precision upper bound) )
    '''

    labeled_pairs.drop_duplicates(inplace=True)
    labeled_pairs.columns = ['id1', 'id2', 'label']
    reduced_cands.columns = ['id1', 'id2']
    reduced_cand_set = set(zip(reduced_cands.id1, reduced_cands.id2))
    predicted_matches = set(zip(predicted_matches.id1, predicted_matches.id2))
    
    # estimate the recall
    # number of positives in the labeled sample
    actual_pos = float(labeled_pairs.label.sum())
    # the maximum number of postives in the candidate set
    max_actual_pos = float(actual_pos + len(reduced_cand_set) - len(labeled_pairs))
    
    # true positives in the labeled sample
    true_pos = float(labeled_pairs.apply(lambda x : (x['id1'], x['id2']) in predicted_matches and x['label'], axis=1).sum())
    #estimated recall
    recall = float(true_pos / actual_pos)

    recall_error = Z * sqrt( ((recall * (1 - recall)) / (actual_pos)) * ((max_actual_pos - actual_pos) / (max_actual_pos - 1)) )


    # estimate Precision
    labeled_set  = set(zip(labeled_pairs.id1, labeled_pairs.id2))
    predicted_pos = float(len(labeled_set & predicted_matches))
    
    predicted_pos_in_reduced_cand_set = float(len(reduced_cand_set & predicted_matches))
    
    alpha =  predicted_pos_in_reduced_cand_set / len(predicted_matches)
    precision = alpha * (true_pos / predicted_pos)
    
    precision_error = alpha * Z * sqrt( ((precision * (1 - precision)) / predicted_pos) * (float((len(predicted_matches) - predicted_pos)) / (len(predicted_matches)  - 1)) )

    return ((recall - recall_error, recall + recall_error),
            (precision - precision_error, precision + precision_error))

# [NEED MODIFICATION: Modify this cell to point to the file location]

In [8]:
# read the labeled pairs file, i.e. the file with the labels
labeled_pairs = pd.read_csv('./result400.csv')
print(estimate_PR(labeled_pairs, dfc, dfp))

((1.0, 1.0), (0.9793995474349522, 1.0020819340465292))


# Alternatively if you run into issues running this script on your laptop you can use
# https://colab.research.google.com/notebooks/welcome.ipynb